This script loads behavioral mice data (from `biasedChoiceWorld` sessions and, separately, the last three sessions of training) only from mice that pass a given (stricter) training criterion defined in a *schema* uploaded by Anne Urai.
The data are slightly reformatted and saved as `.csv` files.

In [1]:
import datajoint as dj
dj.config['database.host'] = 'datajoint.internationalbrainlab.org'

from ibl_pipeline import subject, acquisition, action, behavior, reference, data
from ibl_pipeline.analyses.behavior import PsychResults
from ibl_pipeline.utils import psychofit as psy
from ibl_pipeline.analyses import behavior as behavioral_analyses

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

Please enter DataJoint username: lacerbi
Please enter DataJoint password: ········
Connecting lacerbi@datajoint.internationalbrainlab.org:3306


In [2]:
import os
myPath = r"C:\Users\Luigi\Documents\GitHub\ibl-changepoint\data" # Write here your data path
os.chdir(myPath)

In [20]:
# Get list of mice that satisfy given training criteria (from Anne Urai's analysis)

analyses = dj.create_virtual_module('analyses', 'group_shared_anneurai_analyses')

sess = ((acquisition.Session) *
        acquisition.Session.proj(session_date = 'DATE(session_start_time)'))*analyses.TrainingStatus()*\
subject.Subject.proj('subject_nickname')  & 'task_protocol LIKE "%biased%"'

sess_dat = pd.DataFrame(sess.fetch(order_by='subject_nickname, session_start_time'))
mice_names = sess_dat['subject_nickname'].unique()

print(mice_names)

['CSHL_001' 'CSHL_002' 'CSHL_003' 'CSHL_004' 'CSHL_005' 'CSHL_007'
 'CSHL_008' 'CSHL_010' 'CSHL_012' 'CSHL_013' 'CSHL_014' 'CSHL_015'
 'CSK-IBL-003' 'CSK-les-006' 'CSK-les-007' 'CSK-les-008' 'CSK-scan-001'
 'CSK-scan-002' 'CSK-scan-003' 'dop_1' 'dop_3' 'dop_4' 'dop_5' 'dop_6'
 'DY_001' 'DY_002' 'DY_003' 'DY_005' 'DY_006' 'DY_007' 'IBL-T1' 'IBL-T2'
 'IBL-T3' 'IBL-T4' 'IBL_001' 'IBL_002' 'IBL_005' 'IBL_1' 'IBL_10' 'IBL_11'
 'IBL_13' 'IBL_17' 'IBL_34' 'IBL_36' 'IBL_45' 'IBL_46' 'IBL_47'
 'ibl_witten_04' 'ibl_witten_05' 'ibl_witten_06' 'ibl_witten_07'
 'ibl_witten_11' 'ibl_witten_12' 'ibl_witten_14' 'ibl_witten_15' 'KS002'
 'KS003' 'KS004' 'KS005' 'KS014' 'KS015' 'LTC02' 'LTC03' 'NYU-01' 'NYU-02'
 'NYU-04' 'NYU-06' 'NYU-07' 'SWC_001' 'SWC_002' 'SWC_005' 'ZM_1084'
 'ZM_1085' 'ZM_1086' 'ZM_1087' 'ZM_1089' 'ZM_1091' 'ZM_1092' 'ZM_1093'
 'ZM_1095' 'ZM_1097' 'ZM_1098' 'ZM_1367' 'ZM_1369' 'ZM_1371' 'ZM_1372'
 'ZM_1742' 'ZM_1743' 'ZM_1744' 'ZM_1745' 'ZM_1746' 'ZM_1748'
 '_iblrig_test_mouse']


In [13]:
# Only take mice that reach training status 'trained_1b' or 'ready4ephysrig'

temp_mice = list()
for mouse_nickname in mice_names:
    training_status = sess_dat[sess_dat['subject_nickname'] == mouse_nickname]['training_status']    
    good_flag = np.any((training_status == 'trained_1b')  | (training_status == 'ready4ephysrig'))
    if good_flag:
        temp_mice.append(mouse_nickname)

mice_names = temp_mice
print(mice_names)

['CSHL_001', 'CSHL_002', 'CSHL_003', 'CSHL_004', 'CSHL_005', 'CSHL_007', 'CSHL_008', 'CSHL_010', 'CSHL_012', 'CSHL_013', 'CSHL_014', 'CSHL_015', 'CSK-IBL-003', 'CSK-les-006', 'CSK-les-007', 'CSK-les-008', 'CSK-scan-001', 'CSK-scan-002', 'CSK-scan-003', 'dop_1', 'dop_3', 'dop_4', 'dop_5', 'dop_6', 'DY_001', 'DY_002', 'DY_003', 'DY_005', 'DY_006', 'DY_007', 'IBL-T1', 'IBL-T2', 'IBL-T3', 'IBL-T4', 'IBL_001', 'IBL_002', 'IBL_005', 'IBL_1', 'IBL_10', 'IBL_11', 'IBL_13', 'IBL_17', 'IBL_34', 'IBL_36', 'IBL_45', 'IBL_46', 'IBL_47', 'ibl_witten_04', 'ibl_witten_05', 'ibl_witten_06', 'ibl_witten_07', 'ibl_witten_11', 'ibl_witten_12', 'ibl_witten_14', 'ibl_witten_15', 'KS002', 'KS003', 'KS004', 'KS005', 'KS014', 'KS015', 'LTC02', 'LTC03', 'NYU-01', 'NYU-02', 'NYU-04', 'NYU-06', 'NYU-07', 'SWC_001', 'SWC_002', 'SWC_005', 'ZM_1084', 'ZM_1085', 'ZM_1086', 'ZM_1087', 'ZM_1089', 'ZM_1091', 'ZM_1092', 'ZM_1093', 'ZM_1095', 'ZM_1097', 'ZM_1098', 'ZM_1367', 'ZM_1369', 'ZM_1371', 'ZM_1372', 'ZM_1742', 'ZM

In [5]:
sess_train  = (acquisition.Session & 'task_protocol LIKE "%training%"')
sess_stable = (acquisition.Session & 'task_protocol LIKE "%biased%"')
stable_mice_names = list()

# Perform at least this number of sessions
MinSessionNumber = 1 

def get_mouse_data(df):
    position_deg = 35.    # Stimuli appear at +/- 35 degrees

    # Create new dataframe
    datamat = pd.DataFrame()
    datamat['trial_num'] = df['trial_id']
    datamat['session_num'] = np.cumsum(df['trial_id'] == 1)
    datamat['stim_probability_left'] = df['trial_stim_prob_left']
    signed_contrast = df['trial_stim_contrast_right'] - df['trial_stim_contrast_left']
    datamat['contrast'] = np.abs(signed_contrast)
    datamat['position'] = np.sign(signed_contrast)*position_deg
    datamat['response_choice'] = df['trial_response_choice']
    datamat.loc[df['trial_response_choice'] == 'CCW','response_choice'] = 1
    datamat.loc[df['trial_response_choice'] == 'CW','response_choice'] = -1
    datamat.loc[df['trial_response_choice'] == 'No Go','response_choice'] = 0
    datamat['trial_correct'] = np.double(df['trial_feedback_type']==1)
    datamat['reaction_time'] = df['trial_response_time'] - df['trial_stim_on_time'] # double-check

    # Since some trials have zero contrast, need to compute the alleged position separately
    datamat.loc[(datamat['trial_correct'] == 1) & (signed_contrast == 0),'position'] = \
        datamat.loc[(datamat['trial_correct'] == 1) & (signed_contrast == 0),'response_choice']*position_deg
    datamat.loc[(datamat['trial_correct'] == 0) & (signed_contrast == 0),'position'] = \
        datamat.loc[(datamat['trial_correct'] == 0) & (signed_contrast == 0),'response_choice']*(-position_deg)
    
    return datamat

In [6]:
# Loop over all mice
for mouse_nickname in mice_names:
    
    mouse_subject = {'subject_nickname': mouse_nickname}
    
    # Get mouse data for biased sessions
    behavior_stable = (behavior.TrialSet.Trial & (subject.Subject & mouse_subject)) \
        * sess_stable.proj('session_uuid','task_protocol') * subject.Subject.proj('subject_nickname') \
        * subject.SubjectLab.proj('lab_name')
    df = pd.DataFrame(behavior_stable.fetch(order_by='subject_nickname, session_start_time, trial_id'))

    if len(df) > 0:           # The mouse has performed in at least one stable session with biased blocks            
        datamat = get_mouse_data(df)

        # Take mice that have performed a minimum number of sessions
        if np.max(datamat['session_num']) >= MinSessionNumber:

            # Should add 'N' to mice names that start with numbers?

            # Save dataframe to CSV file
            filename = mouse_nickname + '.csv'
            datamat.to_csv(filename,index=False)
            stable_mice_names.append(mouse_nickname)

            # Get mouse last sessions of training data
            behavior_train = (behavior.TrialSet.Trial & (subject.Subject & mouse_subject)) \
                * sess_train.proj('session_uuid','task_protocol') * subject.Subject.proj('subject_nickname') \
                * subject.SubjectLab.proj('lab_name')
            df_train = pd.DataFrame(behavior_train.fetch(order_by='subject_nickname, session_start_time, trial_id'))

            datamat_train = get_mouse_data(df_train)
            Nlast = np.max(datamat_train['session_num']) - 3
            datamat_final = datamat_train[datamat_train['session_num'] > Nlast]

            # Save final training dataframe to CSV file
            filename = mouse_nickname + '_endtrain.csv'
            datamat_final.to_csv(filename,index=False)
            

print(stable_mice_names)

['CSHL_007', 'CSHL_010', 'CSK-les-008', 'DY_006', 'IBL-T4', 'IBL_13', 'IBL_17', 'IBL_36', 'IBL_45', 'ibl_witten_06', 'KS003', 'NYU-01', 'ZM_1092', 'ZM_1745', 'ZM_1746']


In [106]:
sess_dat['training_status'].unique()

array(['trained_1a', 'ready4ephysrig', 'trained_1b', 'intraining'],
      dtype=object)

In [8]:
# Only take mice that reach training status 'trained_1b'

temp_mice = list()
for mouse_nickname in mice_names:
    training_status = sess_dat[sess_dat['subject_nickname'] == mouse_nickname]['training_status'] 
    good_flag = np.any(training_status == 'trained_1b')
    if good_flag:
        temp_mice.append(mouse_nickname)

mice_names = temp_mice
print(mice_names)

['CSHL_010', 'CSK-les-008', 'DY_006', 'IBL-T4', 'IBL_13', 'IBL_17', 'IBL_36', 'IBL_45', 'ibl_witten_06', 'KS003', 'NYU-01', 'ZM_1092', 'ZM_1745', 'ZM_1746']


In [112]:
training_labels = ['intraining','trained_1a','trained_1b','ready4ephysrig']

tab = np.zeros((len(mice_names),len(training_labels)))

for imouse,mouse_nickname in enumerate(mice_names):
    training_status = sess_dat[sess_dat['subject_nickname'] == mouse_nickname]['training_status']
    for idx,tl in enumerate(training_labels):
        a = training_status == tl
        w = np.where(a)
        if len(w[0]) == 0:
            tab[imouse,idx] = -1
        else:
            tab[imouse,idx] = w[0][0]
print(tab)

[[-1.  0. 25. 31.]
 [ 0. -1.  1. -1.]
 [-1.  0.  6.  7.]
 [-1. -1.  0.  3.]
 [-1. -1.  0.  8.]
 [-1. -1.  0. -1.]
 [-1. -1.  0. -1.]
 [-1. -1.  0.  4.]
 [-1. -1.  0.  8.]
 [-1. -1.  0. 25.]
 [ 0.  6. 35. -1.]
 [-1. -1.  0.  4.]
 [-1. -1.  0. -1.]
 [-1. -1.  0.  3.]]


In [105]:
df = pd.DataFrame(columns=training_labels)
tab

array([[-1.,  0., 25., 31.],
       [ 0., -1.,  1., -1.],
       [-1.,  0.,  6.,  7.],
       [-1., -1.,  0.,  3.],
       [-1., -1.,  0.,  8.],
       [-1., -1.,  0., -1.],
       [-1., -1.,  0., -1.],
       [-1., -1.,  0.,  4.],
       [-1., -1.,  0.,  8.],
       [-1., -1.,  0., 25.],
       [ 0.,  6., 35., -1.],
       [-1., -1.,  0.,  4.],
       [-1., -1.,  0., -1.],
       [-1., -1.,  0.,  3.]])

In [84]:
a = training_status == training_labels[1] + 'a'
f = np.where(a)
len(f[0])


0

In [ ]:

train_list = list()
table
for mouse_nickname in mice_names:
    train_list.extend(list(sess_dat[sess_dat['subject_nickname'] == mouse_nickname]['training_status']))
    
    
np.unique(train_list)

In [21]:
sess_dat[sess_dat['subject_nickname'] == 'CSHL_010']['training_status'] 

3470                intraining
3471           ready for ephys
3472            ready4ephysrig
3473           ready4recording
3474                   trained
3475                trained_1a
3476                trained_1b
3477      training in progress
3478               untrainable
3479    wrong session type run
3480                intraining
3481           ready for ephys
3482            ready4ephysrig
3483           ready4recording
3484                   trained
3485                trained_1a
3486                trained_1b
3487      training in progress
3488               untrainable
3489    wrong session type run
3490                intraining
3491           ready for ephys
3492            ready4ephysrig
3493           ready4recording
3494                   trained
3495                trained_1a
3496                trained_1b
3497      training in progress
3498               untrainable
3499    wrong session type run
                 ...          
4020                intraining
4021    

In [17]:
mouse_nickname = mice_names[0]
training_status = sess_dat[sess_dat['subject_nickname'] == mouse_nickname]['training_status']

In [19]:
sess_dat

,subject_uuid,session_start_time,training_status,session_uuid,session_number,session_end_time,session_lab,session_location,task_protocol,session_type,session_narrative,session_ts,session_date,subject_nickname
0,ecba4914-30c3-4a0b-9d65-1ef5d80ca5f2,2019-04-29 12:43:31,intraining,f9de723b-b947-4a68-8ae2-710013a14d5f,1,2019-04-29 14:09:59,churchlandlab,cshl-behavior-bpod2,_iblrig_tasks_biasedChoiceWorld4.0.1,Experiment,None,2019-08-06 21:42:14,2019-04-29,CSHL_001
1,ecba4914-30c3-4a0b-9d65-1ef5d80ca5f2,2019-04-29 12:43:31,ready for ephys,f9de723b-b947-4a68-8ae2-710013a14d5f,1,2019-04-29 14:09:59,churchlandlab,cshl-behavior-bpod2,_iblrig_tasks_biasedChoiceWorld4.0.1,Experiment,None,2019-08-06 21:42:14,2019-04-29,CSHL_001
2,ecba4914-30c3-4a0b-9d65-1ef5d80ca5f2,2019-04-29 12:43:31,ready4ephysrig,f9de723b-b947-4a68-8ae2-710013a14d5f,1,2019-04-29 14:09:59,churchlandlab,cshl-behavior-bpod2,_iblrig_tasks_biasedChoiceWorld4.0.1,Experiment,None,2019-08-06 21:42:14,2019-04-29,CSHL_001
3,ecba4914-30c3-4a0b-9d65-1ef5d80ca5f2,2019-04-29 12:43:31,ready4recording,f9de723b-b947-4a68-8ae2-710013a14d5f,1,2019-04-29 14:09:59,churchlandlab,cshl-behavior-bpod2,_iblrig_tasks_biasedChoiceWorld4.0.1,Experiment,None,2019-08-06 21:42:14,2019-04-29,CSHL_001
4,ecba4914-30c3-4a0b-9d65-1ef5d80ca5f2,2019-04-29 12:43:31,trained,f9de723b-b947-4a68-8ae2-710013a14d5f,1,2019-04-29 14:09:59,churchlandlab,cshl-behavior-bpod2,_iblrig_tasks_biasedChoiceWorld4.0.1,Experiment,None,2019-08-06 21:42:14,2019-04-29,CSHL_001
5,ecba4914-30c3-4a0b-9d65-1ef5d80ca5f2,2019-04-29 12:43:31,trained_1a,f9de723b-b947-4a68-8ae2-710013a14d5f,1,2019-04-29 14:09:59,churchlandlab,cshl-behavior-bpod2,_iblrig_tasks_biasedChoiceWorld4.0.1,Experiment,None,2019-08-06 21:42:14,2019-04-29,CSHL_001
6,ecba4914-30c3-4a0b-9d65-1ef5d80ca5f2,2019-04-29 12:43:31,trained_1b,f9de723b-b947-4a68-8ae2-710013a14d5f,1,2019-04-29 14:09:59,churchlandlab,cshl-behavior-bpod2,_iblrig_tasks_biasedChoiceWorld4.0.1,Experiment,None,2019-08-06 21:42:14,2019-04-29,CSHL_001
7,ecba4914-30c3-4a0b-9d65-1ef5d80ca5f2,2019-04-29 12:43:31,training in progress,f9de723b-b947-4a68-8ae2-710013a14d5f,1,2019-04-29 14:09:59,churchlandlab,cshl-behavior-bpod2,_iblrig_tasks_biasedChoiceWorld4.0.1,Experiment,None,2019-08-06 21:42:14,2019-04-29,CSHL_001
8,ecba4914-30c3-4a0b-9d65-1ef5d80ca5f2,2019-04-29 12:43:31,untrainable,f9de723b-b947-4a68-8ae2-710013a14d5f,1,2019-04-29 14:09:59,churchlandlab,cshl-behavior-bpod2,_iblrig_tasks_biasedChoiceWorld4.0.1,Experiment,None,2019-08-06 21:42:14,2019-04-29,CSHL_001
9,ecba4914-30c3-4a0b-9d65-1ef5d80ca5f2,2019-04-29 12:43:31,wrong session type run,f9de723b-b947-4a68-8ae2-710013a14d5f,1,2019-04-29 14:09:59,churchlandlab,cshl-behavior-bpod2,_iblrig_tasks_biasedChoiceWorld4.0.1,Experiment,None,2019-08-06 21:42:14,2019-04-29,CSHL_001


In [ ]:
analyses.